In [541]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from IPython import display
import seaborn as sns
import csv

In [542]:
df = pd.read_csv('2017/Student_Feedback_Surveys-Superview.csv')

In [543]:
df

In [544]:
df_track = df.groupby('Track').size()

df_track.plot(kind='pie', subplots=True, autopct='%1.1f%%', figsize=(8,8))
plt.title('Pie Chart of Percentage of Students by Track ')
plt.ylabel('')
plt.show()


In [545]:
df['Track'].value_counts().plot(kind='bar')

### Convert 'Rating (Num)' column that is detected as Object into Integer

In [546]:
df['Rating (Num)'] = pd.factorize(df['Rating (Num)'])[0]

In [547]:
df['Rating (Num)'].dtype

In [548]:
promoter_ls = len(df[df['Rating (Num)'] >= 9])
passive_ls = len(df[(df['Rating (Num)'] > 6) & (df['Rating (Num)'] < 9)])
detractor_ls = len(df[df['Rating (Num)'] <= 6])

total = len(df['Rating (Num)'].dropna().values)
# data = {'Promoter (9-10)': promoter_ls, 'Passive (7-8)': passive_ls, 'Detractor (1-6)': detractor_ls}
# pd.DataFrame(data, index = ['Rating (Num)'])

data = {'Percentage': [promoter_ls/total, passive_ls/total, detractor_ls/total]}
nps_df = pd.DataFrame(data, index = ['Promoter (9-10)', 'Passive (7-8)', 'Detractor (1-6)'])
nps_df.round(5)

### NPS = (Promoters - Detractors) / (Promoters + Passives + Detractors)

In [549]:
# import blender_notebook.plot as plots
# plots.barplot(df.crosstab('Track', 'Rating (Num)', w='Weight', pct=True),stacked=False)
df.groupby(['Week','Track'])['Category'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True)

In [550]:
track_ls = list(df['Track'].dropna().sort_values().unique())

data = {}
for track in track_ls:
    ls = []
    ls.append(len(df[(df['Rating (Num)'] >= 9) & (df['Track'] == track)]))
    ls.append(len(df[(df['Rating (Num)'] > 6) & (df['Rating (Num)'] < 9) & (df['Track'] == track)]))
    ls.append(len(df[(df['Rating (Num)'] <= 6) & (df['Track'] == track)]))
    data[track] = ls
    
nps_df = pd.DataFrame(data, index = ['Promoter (9-10)', 'Passive (7-8)', 'Detractor (1-6)'])

nps_df

In [551]:
df['Category'] = ''

df.loc[df['Rating (Num)'] >= 9, 'Category'] = 'Promoter (9-10)'
df.loc[(df['Rating (Num)'] > 6) & (df['Rating (Num)'] < 9), 'Category'] = 'Passive (7-8)'
df.loc[df['Rating (Num)'] <= 6, 'Category'] = 'Detractor (1-6)'
# df
sns.countplot(x='Track', hue='Category', data=df, palette=["#cf1543", "#fff04d", "#274ed9"])

In [552]:
sns.catplot(y='Location', hue='Category', kind='count', data=df, palette=["#cf1543", "#fff04d", "#274ed9"])

## Calculate weekly NPS for each track


### Average NPS by Location

In [553]:
def calculate_weekly_nps_by_location(week, location):
    location_df = df[(df['Week'] == week) & (df['Location'] == location)]
    promoter = len(location_df[location_df['Category'] == 'Promoter (9-10)'])
    passive = len(location_df[location_df['Category'] == 'Passive (7-8)'])
    detractor = len(location_df[location_df['Category'] == 'Detractor (1-6)'])
    try:
        nps = (promoter - detractor) / (promoter + passive + detractor) * 100
    except ZeroDivisionError:  
        nps = float('Nan')
#         nps = 0
    return nps 

In [554]:
week_ls = list(df['Week'].sort_values().unique())
location_ls = list(df['Location'].dropna().sort_values().unique())
d_dict = {}

for week in week_ls:
    nps_ls = []
    for location in location_ls:
        nps_ls.append(calculate_weekly_nps_by_location(week, location))
    d_dict[week] = nps_ls
    
d_f = pd.DataFrame(d_dict, index=location_ls)
d_f.round(2)


In [555]:
df_each_location = ({'x1': d_f.iloc[0],
                     'x2': d_f.iloc[1],
                     'x3': d_f.iloc[2],
                     'x4': d_f.iloc[3],
                     'x5': d_f.iloc[4],
                     'y': range(1,9)})

fig, axes = plt.subplots(nrows=4, ncols=5, sharex=True, sharey='row', figsize=(10,10))
axes_list = [item for sublist in axes for item in sublist] 

for countryname, selection in df_each_location:
    ax = axes_list.pop(0)
    selection.plot(x='Year', y='GDP_per_capita', label=countryname, ax=ax, legend=False, clip_on=False)
    ax.set_title(countryname)
    ax.tick_params(
        which='both',
        bottom='off',
        left='off',
        right='off',
        top='off'
    )
    ax.grid(linewidth=0.25)
    ax.set_xlim((1950, 2020))
    ax.set_xlabel("")
    ax.set_xticks(range(1950, 2015, 25))
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Now use the matplotlib .remove() method to 
# delete anything we didn't use
for ax in axes_list:
    ax.remove()

plt.tight_layout()

In [ ]:
def calculate_weekly_nps(week, track):
    track_df = df[(df['Week'] == week) & (df['Track'] == track)]
    promoter = len(track_df[track_df['Category'] == 'Promoter (9-10)'])
    passive = len(track_df[track_df['Category'] == 'Passive (7-8)'])
    detractor = len(track_df[track_df['Category'] == 'Detractor (1-6)'])
    try:
        nps = (promoter - detractor) / (promoter + passive + detractor) * 100
    except ZeroDivisionError:  
        nps = float('Nan')
#         nps = 0
    return nps    

In [ ]:
week_ls = list(df['Week'].sort_values().unique())
data_dict = {}

for week in week_ls:
    nps_ls = []
    for track in track_ls:
        nps_ls.append(calculate_weekly_nps(week, track))
    data_dict[week] = nps_ls
    
data_f = pd.DataFrame(data_dict, index=track_ls)
data_f.round(2)


In [ ]:
df_each_track = ({'x1': data_f.iloc[0],
                 'x2': data_f.iloc[1],
                 'x3': data_f.iloc[2],
                 'x4': data_f.iloc[3],
                 'x5': data_f.iloc[4],
                 'y': range(1,9)})

plt.plot('y', 'x1', data=df_each_track, marker='', color='red', linewidth=2)
plt.plot('y', 'x2', data=df_each_track, marker='', color='blue', linewidth=2)
plt.plot('y', 'x3', data=df_each_track, marker='', color='green', linewidth=2)
plt.plot('y', 'x4', data=df_each_track, marker='', color='orange', linewidth=2)
plt.plot('y', 'x5', data=df_each_track, marker='', color='gray', linewidth=2)
plt.title('Weeky NPS by Track in 2017')
plt.xlabel('Week')
plt.ylabel('NPS Score')
plt.legend()


In [ ]:
df[df['Track'] == 'Games'].sort_values('Week')


In [ ]:
# g = sns.catplot(x='Track', col='deck', col_wrap=4,
#                data=nps_df, kind='count')

# fig.subtitle('sf')
# plt.show()

### Reason students enter low rate

In [ ]:
pace_df = df[df['Category'] == 'Detractor (1-6)']
g = sns.countplot(x='Schedule Pacing', hue='Category', data=pace_df, palette=("Blues_d"))
# g.set_plt.xticks(rotation=45)
g.set_xticklabels(g.get_xticklabels(), rotation=30)

### Schedule pacing by track

In [ ]:
sns.countplot(x='Track', hue='Schedule Pacing', data=pace_df, palette=["#cf1543", "#fff04d", "#274ed9", "#b549e3", "#49e34e"])
plt.title('Schedule Pacing by Track')


In [ ]:
len(df['ID'].unique())

### Track Taken by Location


### Average NPS by Week


In [ ]:
ave_ls = []
for week in week_ls:
    ave_ls.append(data_f[week].mean())
    
ave_nps = pd.DataFrame({'a': ave_ls,
                       'y': range(1,9)})
    
plt.plot('y', 'a', data=ave_nps, marker='', color='orange', linewidth=2)



In [ ]:
# sum all the rating not counts
df[df['Week'] == 'Week 2'].groupby('Category').value_counts()


In [ ]:
df[df['Week'] == 'Week 2'].sort_values('Category')


In [540]:
df.head()